In [1]:
import numpy as np
import pandas as pd
import sys

sys.path.append('../src/')
from preprocess import Weather


import itertools
from datetime import timedelta

## ダウンロードしたデータの処理

In [2]:
download_df = pd.read_csv('../data/download_weather.csv', encoding='SHIFT-JIS', index_col=0, header=[0,1,2])
download_df.head()

神奈川                                                          \
               横浜                                                           
          平均気温(℃) 最高気温(℃) 最高気温(℃).1 最低気温(℃) 最低気温(℃).1 降水量の合計(mm) 日照時間(時間)   
2022年5月1日    13.3    16.5     10:38    11.3     19:18         22      0.0   
2022年5月2日    14.2    19.9     14:19    11.3      5:04          3      4.0   
2022年5月3日    15.3    20.2     15:09    11.1      6:00         --     12.3   
2022年5月4日    18.3    23.7     16:01    12.1      5:18         --     12.8   
2022年5月5日    19.8    25.2     14:07    15.5      4:31         --     11.2   

                       青森          ...       東京              山形          \
                       青森          ...       東京              山形           
          平均湿度(％) 平均気温(℃) 最高気温(℃)  ... 日照時間(時間) 平均湿度(％) 平均気温(℃) 最高気温(℃)   
2022年5月1日      82    10.1    13.8  ...      0.0      83     8.0     8.9   
2022年5月2日      70     9.8    15.0  ...      4.1      74    10.5    16.8   
2022年5月3日      64    10.9    16.1  ...     12.3      63    10.3    15.6   
2022年5月4日      58    14.3    20.1  ...     12.4      60    16.4    24.7   
2022年5月5日      61    12.6    17.8  ...     11.8      65    18.5    28.7   

                                                                   
                                                                   
          最高気温(℃).1 最低気温(℃) 最低気温(℃).1 降水量の合計(mm) 日照時間(時間) 平均湿度(％)  
2022年5月1日     17:45     6.3      4:58       14.5      0.0      88  
2022年5月2日     14:01     6.2      5:21         --      6.5      62  
2022年5月3日     14:55     5.5      4:34          0      8.5      57  
2022年5月4日     14:21     6.7      4:48         --     12.9      45  
2022年5月5日     15:22     8.5      5:02         --     12.8      49  

[5 rows x 256 columns]

In [3]:
df = pd.melt(download_df.T.reset_index(), id_vars=['level_0', 'level_1', 'level_2'], var_name="date")
df.rename(columns={'level_0' : 'area', 'level_1' : 'old_area', 'level_2' : 'target'}, inplace=True)
df.date = pd.to_datetime(df.date, format='%Y年%m月%d日')

target_dict = {
    '平均気温(℃)' : 'mean_temp',
    '平均湿度(％)' : 'mean_humid',
    '日照時間(時間)' : 'sun_time',
    '最低気温(℃)' : 'min_temp',
    '最低気温(℃).1' : 'min_temp_time',
    '最高気温(℃)' : 'max_temp',
    '最高気温(℃).1' : 'max_temp_time',
    '降水量の合計(mm)' : 'sum_rain'
    }

df.target.replace(target_dict, inplace=True)


target = pd.DataFrame(
    {
        name : 
            group.drop('target', axis=1)
            .rename(columns={'value' : name})
            .sort_values(by = ['area', 'date'])[name]
            .reset_index(drop=True)
            .to_list()
        for name, group in df.groupby('target')
    }
)

out = pd.concat(
    [
        df[['area', 'old_area', 'date']].drop_duplicates().reset_index(drop=True),
        target
        ], 
    axis =1
    )

# 降水量のデータを変換
out.sum_rain = out.sum_rain.str.replace('--', '0').astype('float32')

# 地域名の変換用の辞書：’市区町村’ -> '都道府県'
area_name_dict = out[['area', 'old_area']].set_index('old_area').to_dict()['area']

In [4]:
weather = pd.read_csv('../data/weather.csv')
weather.area = weather.area.replace(area_name_dict)
weather.date = pd.to_datetime(weather.date.astype(str))

all_weather =pd.concat(
    [
        weather.drop(['max_temp_time', 'min_temp_time'], axis = 1),
        out.drop('old_area', axis=1).drop(['max_temp_time', 'min_temp_time'], axis = 1)
    ]
).reset_index(drop=True)

all_weather.area.replace({'十勝' : '北海道'}, inplace=True)


# 順番を入れ替え
all_weather.sort_values(['area', 'date'], inplace=True)

# インデックス
all_weather.reset_index(drop = True, inplace = True)
all_weather.set_index('date', inplace=True)

# 新しく保存
all_weather.to_csv('../data/all_weather.csv', encoding='SHIFT-JIS')

# 読み込み
read_all_weather = pd.read_csv('../data/all_weather.csv', encoding='SHIFT-JIS', index_col=0)

In [5]:
all_weather.head()

,mean_temp,max_temp,min_temp,sum_rain,sun_time,mean_humid,area
date,,,,,,,
2004-11-06,15.8,21.5,10.2,0.0,8.5,70.0,佐賀
2004-11-07,16.3,23.0,11.4,0.0,8.5,72.0,佐賀
2004-11-08,16.8,23.7,11.2,0.0,9.1,68.0,佐賀
2004-11-09,18.2,25.6,11.7,0.0,9.3,61.0,佐賀
2004-11-10,19.1,23.9,13.4,2.5,0.1,69.0,佐賀


## 訓練データとテストデータの前処理

In [6]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

area =set(itertools.chain.from_iterable(train.area.drop_duplicates().str.split('_').to_list()))
print('-------- 気象情報と訓練データのエリア情報の整合性の確認 --------')
print('訓練データにあって気象情報のエリアに無いもの  :', area - set(all_weather.area))
print('気象情報にあって訓練データのエリアに無いもの :', set(all_weather.area) - area)

-------- 気象情報と訓練データのエリア情報の整合性の確認 --------
訓練データにあって気象情報のエリアに無いもの  : {'中国', 'メキシコ', 'ニュージーランド', 'トンガ', 'アメリカ', 'カナダ', '各地'}
気象情報にあって訓練データのエリアに無いもの : set()


In [7]:
# テストデータと訓練データの連結
df = pd.concat([train, test]).sort_values(['kind', 'date']).reset_index(drop=True)
df.date = pd.to_datetime(df['date'].astype(str))


for kind in set(train.kind) & set(test.kind):
    sub_df = df[df.kind == kind]
    print(kind + '  :', df[df.kind == kind].date.min())

にんじん  : 2005-11-07 00:00:00
なましいたけ  : 2005-11-07 00:00:00
ねぎ  : 2005-11-07 00:00:00
ミニトマト  : 2015-05-01 00:00:00
キャベツ  : 2005-11-07 00:00:00
だいこん  : 2005-11-07 00:00:00
セルリー  : 2005-12-01 00:00:00
そらまめ  : 2006-04-01 00:00:00
ほうれんそう  : 2005-11-07 00:00:00
こまつな  : 2005-11-07 00:00:00
レタス  : 2005-11-07 00:00:00
ピーマン  : 2005-11-07 00:00:00
じゃがいも  : 2005-11-07 00:00:00
はくさい  : 2005-11-07 00:00:00
きゅうり  : 2005-11-07 00:00:00
トマト  : 2005-11-07 00:00:00


In [8]:
print('実際に利用するエリア情報')
target_area = set(itertools.chain.from_iterable(sub_df.area.drop_duplicates().str.split('_').to_list()))
print(target_area)

実際に利用するエリア情報
{'栃木', '群馬', '愛知', '茨城', '熊本', '青森', '福島', '千葉', '北海道', '各地'}


## 気象情報の特徴量の作成

In [9]:
# 全国情報を作成する
all_weather = all_weather[all_weather.area.isin(target_area)]
average_all_area =  all_weather.reset_index().drop('area', axis = 1).groupby('date').mean()
average_all_area['area'] = '各地'
all_area_agg_weather = pd.concat([average_all_area, all_weather])

# 気象情報が存在する全日程
all_date = set(all_area_agg_weather.index.drop_duplicates())

target_weather_info = pd.concat(
    [
        group.drop('area', axis = 1).rename(columns = lambda x: name + '_' + x)
        for name, group 
        in all_area_agg_weather.groupby('area')
        ], 
    axis=1
    )

target_weather_info.head()

,北海道_mean_temp,北海道_max_temp,北海道_min_temp,北海道_sum_rain,北海道_sun_time,北海道_mean_humid,千葉_mean_temp,千葉_max_temp,千葉_min_temp,千葉_sum_rain,...,茨城_min_temp,茨城_sum_rain,茨城_sun_time,茨城_mean_humid,青森_mean_temp,青森_max_temp,青森_min_temp,青森_sum_rain,青森_sun_time,青森_mean_humid
date,,,,,,,,,,,,,,,,,,,,,
2004-11-06,8.5,9.5,7.3,0.5,0.0,89.0,17.4,21.2,14.1,0.0,...,10.9,0.0,5.9,83.0,13.1,20.1,8.1,5.0,1.6,77.0
2004-11-07,8.1,12.3,4.6,0.0,9.4,55.0,17.0,22.2,13.5,0.0,...,9.4,0.0,8.4,80.0,9.6,12.5,5.4,0.0,4.1,63.0
2004-11-08,5.3,12.0,0.3,0.0,9.1,68.0,15.8,19.0,12.7,0.0,...,9.5,0.0,4.0,79.0,9.0,15.5,3.2,0.0,8.9,72.0
2004-11-09,7.3,17.2,0.1,0.0,8.5,74.0,16.8,21.5,12.4,0.0,...,7.2,0.0,8.5,79.0,12.2,17.7,5.7,1.5,1.2,85.0
2004-11-10,7.8,12.5,4.5,0.0,5.7,81.0,16.7,21.9,11.5,0.0,...,7.8,0.0,8.0,82.0,11.6,16.4,6.9,0.0,7.5,89.0


In [10]:
roll = 7
lag = 10

test_df = target_weather_info.rolling(roll).mean()
test_df.set_index(target_weather_info.index + timedelta(days=lag))
test_df.rename(columns=lambda x:x + '_roll_' + str(roll) + '_lag_' + str(lag), inplace=True)
test_df = test_df[test_df.index <= max(target_weather_info.index)]

In [11]:
res = pd.concat([test_df, target_weather_info], axis = 1)

---

# クラスの処理について

## 前処理クラスの試行

In [12]:
sys.path.append('../src/')
from preprocess import Preprocess

import pandas as pd
import itertools

In [13]:
tester = Preprocess()

# 訓練データとテストデータの読み込み開始
tester.read_train_test_from_csv()

# 前処理開始
tester.preprocess()

In [14]:
tester.preprocessed_data

{'にんじん': {'train':             mode_price  北海道_mean_temp  北海道_max_temp  北海道_min_temp  \
  date                                                                
  2005-11-07       892.5           14.6          20.5          10.2   
  2005-11-08      1155.0           10.4          14.0           5.8   
  2005-11-10      1050.0            3.8           9.4          -1.6   
  2005-11-11      1050.0            4.7           9.5          -0.1   
  2005-11-12       787.5            4.9           8.6           1.4   
  ...                ...            ...           ...           ...   
  2022-04-25      1080.0           13.1          22.9           5.0   
  2022-04-26      1512.0            8.9          15.4           2.0   
  2022-04-28      1404.0           11.3          15.9           7.3   
  2022-04-29      1512.0            9.5          15.7           1.1   
  2022-04-30      1512.0           10.3          17.0           2.8   
  
              北海道_sum_rain  北海道_sun_time  北海道_mean_humid 

## weather class の試行

In [15]:
sys.path.append('../src/')
from preprocess import Weather

import pandas as pd
import itertools

In [16]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

# テストデータと訓練データの連結
df = pd.concat([train, test]).sort_values(['kind', 'date']).reset_index(drop=True)
df.date = pd.to_datetime(df['date'].astype(str))


for kind in set(train.kind) & set(test.kind):
    sub_df = df[df.kind == kind]
    target_area = set(itertools.chain.from_iterable(sub_df.area.drop_duplicates().str.split('_').to_list()))
    print(kind + '  :', df[df.kind == kind].date.min())
    


weather = Weather()
weather.read_from_csv()
res2 = weather.add_feature(target_area)

にんじん  : 2005-11-07 00:00:00
なましいたけ  : 2005-11-07 00:00:00
ねぎ  : 2005-11-07 00:00:00
ミニトマト  : 2015-05-01 00:00:00
キャベツ  : 2005-11-07 00:00:00
だいこん  : 2005-11-07 00:00:00
セルリー  : 2005-12-01 00:00:00
そらまめ  : 2006-04-01 00:00:00
ほうれんそう  : 2005-11-07 00:00:00
こまつな  : 2005-11-07 00:00:00
レタス  : 2005-11-07 00:00:00
ピーマン  : 2005-11-07 00:00:00
じゃがいも  : 2005-11-07 00:00:00
はくさい  : 2005-11-07 00:00:00
きゅうり  : 2005-11-07 00:00:00
トマト  : 2005-11-07 00:00:00


In [17]:
# 訓練データとテストデータに分割
train_test_df = pd.concat(
    [sub_df.set_index('date'), res2], 
    axis = 1
    ).drop(['kind', 'amount', 'area'], axis = 1)

train_input = train_test_df.dropna(axis = 0)
test_input = train_test_df[train_test_df.index > max(train_input.index)]

In [18]:
train_input

,mode_price,北海道_mean_temp,北海道_max_temp,北海道_min_temp,北海道_sum_rain,北海道_sun_time,北海道_mean_humid,千葉_mean_temp,千葉_max_temp,千葉_min_temp,...,茨城_min_temp_roll_7_lag_10,茨城_sum_rain_roll_7_lag_10,茨城_sun_time_roll_7_lag_10,茨城_mean_humid_roll_7_lag_10,青森_mean_temp_roll_7_lag_10,青森_max_temp_roll_7_lag_10,青森_min_temp_roll_7_lag_10,青森_sum_rain_roll_7_lag_10,青森_sun_time_roll_7_lag_10,青森_mean_humid_roll_7_lag_10
date,,,,,,,,,,,,,,,,,,,,,
2005-11-07,2415.0,14.6,20.5,10.2,24.0,2.5,74.0,18.8,24.4,14.7,...,7.371429,7.142857,6.485714,76.285714,13.228571,18.857143,8.300000,1.428571,5.885714,72.142857
2005-11-08,2310.0,10.4,14.0,5.8,4.5,6.8,58.0,17.7,22.0,13.7,...,7.785714,7.142857,6.428571,74.428571,12.885714,18.385714,7.714286,3.857143,5.714286,73.000000
2005-11-10,1995.0,3.8,9.4,-1.6,0.0,7.3,57.0,13.6,18.0,9.8,...,7.385714,6.928571,7.028571,71.714286,10.957143,15.785714,6.400000,8.285714,4.942857,72.428571
2005-11-11,1890.0,4.7,9.5,-0.1,3.0,4.0,83.0,12.5,14.7,9.5,...,6.971429,6.928571,6.271429,71.714286,10.314286,15.014286,5.900000,8.500000,4.314286,73.142857
2005-11-12,1785.0,4.9,8.6,1.4,0.0,7.2,57.0,13.7,19.3,10.9,...,6.414286,8.214286,5.528571,70.857143,9.642857,13.842857,5.685714,8.500000,3.214286,70.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-04-25,1296.0,13.1,22.9,5.0,0.0,11.5,45.0,19.5,25.9,14.7,...,11.671429,4.071429,6.528571,73.285714,12.742857,18.757143,6.271429,0.071429,9.571429,56.571429
2022-04-26,1188.0,8.9,15.4,2.0,0.0,3.8,76.0,21.6,24.1,17.9,...,12.671429,4.642857,5.128571,75.857143,13.157143,18.985714,6.928571,0.500000,7.814286,61.571429
2022-04-28,1188.0,11.3,15.9,7.3,0.0,12.1,24.0,16.3,18.9,14.0,...,13.742857,4.071429,5.371429,72.714286,12.842857,17.785714,8.200000,0.500000,7.657143,64.285714
